In [1]:
import numpy as np
import scipy
import math
from scipy import optimize
import copy
import time

In [2]:
import matlab.engine

In [3]:
mat_eng = matlab.engine.start_matlab()
tf = mat_eng.isprime(37)
print(tf)

True


In [4]:
class Particle(object):
    def __init__(self, position, m=1.0, v=0,v_direction=(0, 0, 0)):
        x, y, z = position
        self.x = x
        self.y = y
        self.z = z
        self.mass = m
        self.velocity = v
        self.vx, self.vy, self.vz = v_direction
    
    def distance(self, otherParticle):
        return
    
    @staticmethod
    def distance(particle1, particle2):
        dx = particle1.x - particle2.x
        dy = particle1.y - particle2.y
        dz = particle1.z - particle2.z
        l2norm = (dx*dx + dy*dy + dz*dz)**((0.5))
        return l2norm
    

In [5]:
def nelder_mead(f, x_start,
                step=(0.1), no_improve_thr=(10e-6),
                no_improv_break=(5), max_iter=(0),
                alpha=(1.), gamma=(2.), rho=(-0.5), sigma=(0.5)):
    '''
        @param f (function): function to optimize, must return a scalar score
            and operate over a numpy array of the same dimensions as x_start
        @param x_start (numpy array): initial position
        @param step (float): look-around radius in initial step
        @no_improv_thr,  no_improv_break (float, int): break after no_improv_break iterations with
            an improvement lower than no_improv_thr
        @max_iter (int): always break after this number of iterations.
            Set it to 0 to loop indefinitely.
        @alpha, gamma, rho, sigma (floats): parameters of the algorithm
            (see Wikipedia page for reference)
        return: tuple (best parameter array, best score)
    '''

    # init
    dim = len(x_start)
    prev_best = f(x_start)
    no_improv = 0
    res = [[x_start, prev_best]]

    for i in range(dim):
        x = copy.copy(x_start)
        x[i] = x[i] + step
        score = f(x)
        res.append([x, score])

    # simplex iter
    iters = 0
    while 1:
        # order
        res.sort(key=lambda x: x[1])
        best = res[0][1]

        # break after max_iter
        if max_iter and iters >= max_iter:
            return res[0]
        iters += 1

        # break after no_improv_break iterations with no improvement

        if best < prev_best - no_improve_thr:
            no_improv = 0
            prev_best = best
        else:
            no_improv += 1

        if no_improv >= no_improv_break:
            return res[0]

        # centroid
        x0 = [(0.)] * dim
        for tup in res[:-1]:
            for i, c in enumerate(tup[0]):
                x0[i] += c / (len(res)-1)

        # reflection
        xr = x0 + alpha*(x0 - res[-1][0])
        rscore = f(xr)
        if res[0][1] <= rscore < res[-2][1]:
            del res[-1]
            res.append([xr, rscore])
            continue

        # expansion
        if rscore < res[0][1]:
            xe = x0 + gamma*(x0 - res[-1][0])
            escore = f(xe)
            if escore < rscore:
                del res[-1]
                res.append([xe, escore])
                continue
            else:
                del res[-1]
                res.append([xr, rscore])
                continue

        # contraction
        xc = x0 + rho*(x0 - res[-1][0])
        cscore = f(xc)
        if cscore < res[-1][1]:
            del res[-1]
            res.append([xc, cscore])
            continue

        # reduction
        x1 = res[0][0]
        nres = []
        for tup in res:
            redx = x1 + sigma*(tup[0] - x1)
            score = f(redx)
            nres.append([redx, score])
        res = nres

In [6]:
class System(object):
    def __init__(self, n=2):
        self.particles = []
        self.zero = None
        for i in range(n):
            self.add(Particle((0,0,0)))
        return
    
    def add(self, particle):
        assert type(particle) == Particle
        particle.x = (particle.x)
        particle.y = (particle.y)
        particle.z = (particle.z)
        self.particles.append(particle)
    
    @staticmethod
    def Lennard_Jones(r):
        return r**(-12) - 2*r**(-6)
    
    @staticmethod
    def potentialEnergyTwoParticles(particle1, particle2):
        r = Particle.distance(particle1, particle2)
        return System.Lennard_Jones(r)
    
    def particlesAsParams(self):
        para = []
        for p in self.particles:
            para.append(p.x)
            para.append(p.y)
            para.append(p.z)
        para = np.array(para)
        return para
    
    def particlesFromParams(self, para):
        assert len(para) % 3 == 0 and len(para) > 0
        i = 0
        for _idx, par in enumerate(para):
            if _idx % 3 == 0:
                self.particles[i].x = par
            if _idx % 3 == 1:
                self.particles[i].y = par
            if _idx % 3 == 2:
                self.particles[i].z = par
                i += 1
        return
    
    def systemPotentialEnergyAsFunc(self, params):
        self.particlesFromParams(params)
        return self.potentialEnergy()

    def systemPotentialEnergyGradient(self, params):
        return optimize.approx_fprime(params, self.systemPotentialEnergyAsFunc, [1e-9]*len(params))

    def potentialEnergy(self):
        p_E = (0.0)
        '''calculate the potential of this particle system'''
        for _id1, p1 in enumerate(self.particles):
            for _id2, p2 in enumerate(self.particles):
                if _id1 == _id2:
                    break
                p_E += self.potentialEnergyTwoParticles(p1, p2)
        return p_E        

In [7]:
def random_vector_generator(d, nm=(1.)):
    '''d: dimension of the vector
       nm: l2-norm of the vector
    '''
    while True:
        gauss = np.random.normal(size=d)
        gauss.dtype
        length = (np.linalg.norm(gauss))
        if length == 0.0:
            continue
        else:
            # x = np.multiply(gauss, nm / length)
            x=gauss
            yield x 

In [8]:
def gradientDes(func, x_start, lr=0.005, max_tol_iter=10, max_iter=1000):
    lr = np.abs(lr)
    tol_iter = 0
    params = np.array(x_start, dtype='float64')
    best_params = params
    cur_y = func(params)
    total_iter = 0
    while tol_iter < 10 and total_iter < max_iter:
        total_iter += 1
        grad = optimize.approx_fprime(params, func, [1e-5]*len(params))
        grad /= np.linalg.norm(grad)
        params -= lr * grad
        tmp_y = func(params)
        if tmp_y < cur_y:
            cur_y = tmp_y
            best_params = params
            tol_iter = 0
        else:
            tol_iter += 1
    return cur_y

In [9]:
def gradSearch(func, x_start, use_scipylib=False, max_iter=100, tol_max_iter=5):
    mn_res = float('inf')
    tol_iter = 0
    param = None
    total_iter = 0
    while tol_iter < tol_max_iter and total_iter < max_iter:
        total_iter += 1
        tmp = gradientDes(func, x_start)
        if tmp < mn_res:
            tol_iter = 0
            mn_res = tmp
            param = tmp
        else:
            tol_iter += 1
        x_start = next(random_vector_generator(len(x_start),np.linalg.norm(x_start)))
    return param

In [10]:
def nelderSearch(func, x_start, n_iters=10000,use_scipylib=False):
    mn_res = float('inf')
    tol_iter = 0
    param = None
    while tol_iter < 5:
        if not use_scipylib:
            tmp = nelder_mead(func, x_start)[1]
        else:
            tmp = optimize.minimize(func, x_start, method='nelder-mead',
               options={'xtol': 1e-8, 'disp': False})['fun']
        if tmp < mn_res-1e-6:
            tol_iter = 0
            mn_res = tmp
            param = tmp
        else:
            tol_iter += 1
        x_start = next(random_vector_generator(len(x_start),np.linalg.norm(x_start)))
    return param

In [11]:
def fminuncSearch(x_start, n_iters=1000, tol_max_iter=10, use_grad=False):
    mn_res = float('inf')
    tol_iter = 0
    param = None
    x_start = next(random_vector_generator(len(x_start),np.linalg.norm(np.random.randint(5))))
    while tol_iter < tol_max_iter:
        x_start = [float(_x) for _x in x_start]
        if use_grad == False:
            x = mat_eng.minPotentialEnergyWithoutGrad(x_start)
        else:
            x = mat_eng.minPotentialEnergyWithGrad(x_start)            
        tmp = mat_eng.potentialEnergy(x)
        if tmp < mn_res-1e-6:
            tol_iter = 0
            mn_res = tmp
            param = tmp
            if tmp < mn_res - 1e-3:
                print(tmp)
        else:
            tol_iter += 1
        x_start = next(random_vector_generator(len(x_start),np.linalg.norm(np.random.randint(5))))
    return param

In [12]:
system = System(5)
first_guess = system.particlesAsParams()

In [13]:
fminuncSearch(first_guess, tol_max_iter=1, use_grad=True)

-9.103852414428367

In [67]:
gradSearch(system.systemPotentialEnergyAsFunc, first_guess,tol_max_iter=100)

/home/jeff/anaconda3/envs/workvenv35/lib/python3.5/site-packages/ipykernel/__main__.py:18: RuntimeWarning: divide by zero encountered in double_scalars
/home/jeff/anaconda3/envs/workvenv35/lib/python3.5/site-packages/ipykernel/__main__.py:18: RuntimeWarning: invalid value encountered in double_scalars


-12.302927110635657

In [66]:
nelderSearch(system.systemPotentialEnergyAsFunc, first_guess, use_scipylib=True)

-16.24370770692708

In [115]:
'''3.098832937043404'''

'3.098832937043404'